In [164]:
# Libraries
from bs4 import BeautifulSoup, Comment
import numpy as np
import pandas as pd
import urllib.request
import requests

## Game Objects

In [165]:
class GameInfo:
    def __init__(self, home_team, away_team, home_spread, away_spread, over_under=None):
        self.home_team = home_team
        self.away_team = away_team
        self.home_spread = home_spread
        self.away_spread = away_spread
        self.over_under = over_under

In [166]:
class GameResults:
    def __init__(self, home_team, away_team, home_score, away_score):
        self.home_team = home_team
        self.away_team = away_team
        self.home_score = home_score
        self.away_score = away_score
        self.total = home_score + away_score
        self.winner = home_team if home_score > away_score else away_team if away_score > home_score else 'tie' 

## Game Info scrape

In [180]:
def get_game_info(score_box):
    odds = score_box.find_all('td', {'class': 'in-progress-odds'})
    over_under = float(odds[0].text.strip()[1:])
    home_line = float(odds[1].text.strip())
    teams = score_box.find_all('td', {'class': 'team'})
    away_team = teams[0].find('a')['href'].split('/')[5]
    home_team = teams[1].find('a')['href'].split('/')[5]
    game_info_obj = GameInfo(home_team, away_team, home_line, home_line * -1, over_under)
    return(game_info_obj)

In [181]:
week_num = 3
link = 'https://www.cbssports.com/nfl/scoreboard/all/2021/regular/' + str(week_num) + '/'
with urllib.request.urlopen(link) as url:
    page = url.read()
soup = BeautifulSoup(page, "html.parser")
# list of game info boxes for all games
score_boxes = soup.find_all('div', {'class':'live-update'})
game_info_list = []
for score_box in score_boxes:
    #print(score_box)
    game_info_list.append(get_game_info(score_box))

In [182]:
# test
print(game_info_list[6].home_team)
print(game_info_list[6].away_team)
print(game_info_list[6].home_spread)
print(game_info_list[6].away_spread)
print(game_info_list[6].over_under)

TEN
IND
-3.0
3.0
52.0


## Game Result scrape

In [177]:
def get_game_results(t):
    teams = t.find_all('a', {'class': 'team'})
    away_team = teams[0]['href'].split('/')[5]
    home_team = teams[1]['href'].split('/')[5]
    scores = t.find_all('td', {'class': 'total-score'})
    game_result_obj = GameResults(home_team, away_team, int(scores[1].text), int(scores[0].text))
    return(game_result_obj)

In [178]:
week_num = 1
link = 'https://www.cbssports.com/nfl/scoreboard/all/2021/regular/' + str(week_num) + '/'
with urllib.request.urlopen(link) as url:
    page = url.read()
soup = BeautifulSoup(page, "html.parser")
t_list = soup.find_all('div', {'class': 'live-update'})
game_result_list = []
for t in t_list:
    game_result_list.append(get_game_results(t))

In [179]:
# test
print(game_result_list[6].home_team)
print(game_result_list[6].away_team)
print(game_result_list[6].home_score)
print(game_result_list[6].away_score)
print(game_result_list[6].total)
print(game_result_list[6].winner)

ATL
PHI
6
32
38
PHI
